In [2]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *


import numpy as np
from langdetect import detect
import translitcodec
import codecs

In [6]:
url = "https://datasets.imdbws.com/"
filenames = ["name.basics.tsv.gz",
             "title.akas.tsv.gz",
             "title.basics.tsv.gz",
             "title.crew.tsv.gz",
             "title.episode.tsv.gz",
             "title.principals.tsv.gz",
             "title.ratings.tsv.gz"
]

output_paths = download_file(url, filenames, "data/")

# Pierwszy rzut oka na dane

Pierwsza ramka:

In [3]:
path_to_file = "data/name.basics.tsv.gz"
df = load_to_dataframe(path_to_file)

In [4]:
df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,<NA>,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


Druga ramka:

In [2]:
path_to_file = "data/title.akas.tsv.gz"
df = load_to_dataframe(path_to_file)

In [3]:
df = df.drop(columns=["language", "types", "attributes"])

,titleId,ordering,title,region,isOriginalTitle
0,tt0000001,1,Carmencita,<NA>,1
1,tt0000001,2,Carmencita,DE,0
2,tt0000001,3,Carmencita,US,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,0
4,tt0000001,5,Καρμενσίτα,GR,0
...,...,...,...,...,...
48657464,tt9916852,7,エピソード #3.20,JP,0
48657465,tt9916852,8,Episodio #3.20,ES,0
48657466,tt9916856,1,The Wind,<NA>,1
48657467,tt9916856,2,The Wind,DE,0


In [4]:
# zamieniamy wszystkie tytuły na lowercase
df["title"] = df["title"].astype(str).str.lower()

In [5]:
df["detectedLang"] = pd.NA

In [10]:
# wspieramy rozpoznawanie języka przez bibliotekę 'langdetect'
def mydetect(x):
    try:
        return detect(x).upper()
    except:
        return pd.NA

In [16]:
df_orig = df.loc[df["isOriginalTitle"] == 1].drop(columns=["region"])

In [18]:
df_orig["detectedLang"] = df_orig["title"].apply(mydetect)

In [11]:
df.loc[df["titleId"] == 'tt0017938']

,titleId,ordering,title,region,isOriginalTitle,detectedLang
72318,tt0017938,1,la glace à trois faces,<NA>,1,<NA>
72319,tt0017938,2,der dreiflügelige spiegel,DE,0,<NA>
72320,tt0017938,3,el espejo de las tres caras,ES,0,<NA>
72321,tt0017938,4,la glace à trois faces,EC,0,<NA>
72322,tt0017938,5,la glace à trois faces,FR,0,<NA>
72323,tt0017938,6,la glace à trois faces,JP,0,<NA>
72324,tt0017938,7,lustro o trzech twarzach,PL,0,<NA>
72325,tt0017938,8,the three-sided mirror,XWW,0,<NA>
72326,tt0017938,9,трельяж,SUHH,0,<NA>


In [8]:
# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[:, ['titleId', 'title', 'region', 'isOriginalTitle']].query(
    "isOriginalTitle == 0"
).merge(
    df.loc[:, ['titleId', 'title', 'isOriginalTitle']].query("isOriginalTitle == 1"),
    on="titleId",
    how="left",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

In [17]:
df_think["detectedLang"] = df_think["title"].apply(lambda x: mydetect(x))

In [ ]:
df_think[df_think["titleId"] == 'tt0017938']

,titleId,title,region
54644,tt0017938,la glace à trois faces,EC
54645,tt0017938,la glace à trois faces,FR
54646,tt0017938,la glace à trois faces,JP


In [10]:
# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
ambig_index = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count().where(
    lambda region: region > 1
).dropna().index

df_with_known_countries = df_think.loc[~df_think.titleId.isin(list(ambig_index))]

In [112]:
df_with_known_countries[df_with_known_countries["region"] == 'EC']

,titleId,title,region
40700,tt0015039,Kinoglaz,EC
42303,tt0015361,Stachka,EC
48902,tt0016847,Faust: Eine deutsche Volkssage,EC
49872,tt0017048,Kurutta ippêji,EC
54644,tt0017938,La glace à trois faces,EC
...,...,...,...
37661612,tt9826286,Kamenák,EC
37752292,tt9876760,80's in the Sand,EC
37758139,tt9880498,El Fútbol del Cerro,EC
37761046,tt9881986,Hentai danchizuma,EC


In [15]:
# utrata danych: 26.6%
1 - df_with_known_countries.shape[0]/df_think.shape[0]

0.2663587633217803

In [18]:
# Teraz do każdego tytułu w 'df' przypisaliśmy kraj, jeśli się da
# jeśli nie, mamy 'NaN' w kolumnie 'region'
df = df.loc[df.isOriginalTitle == 1].loc[:, ["titleId", "title"]].merge(
            df_with_known_countries.loc[:, ["titleId", "region"]],
            how="left",
            on="titleId")

Trzecia ramka:

In [5]:
path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)
df3.head()

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"


In [6]:
df3 = df3.loc[:, ["tconst", "titleType"]]
df3.head()

,tconst,titleType
0,tt0000001,short
1,tt0000002,short
2,tt0000003,short
3,tt0000004,short
4,tt0000005,short


Czwarta ramka:

In [ ]:
path_to_file = "data/title.crew.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

Piąta ramka:

In [ ]:
path_to_file = "data/title.episode.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

Szósta ramka:

In [ ]:
path_to_file = "data/title.principals.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

Siódma ramka:

In [17]:
path_to_file = "data/title.ratings.tsv.gz"
df7 = load_to_dataframe(path_to_file)
df7.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.6,277
2,tt0000003,6.5,2024
3,tt0000004,5.4,179
4,tt0000005,6.2,2791


# Task 1
Quality of movies by country

In [3]:
path_to_file = "data/title.akas.tsv.gz"
df1 = load_to_dataframe(path_to_file)
df1 = df1.drop(columns=["language", "types", "attributes"])

path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)
df3 = df3.loc[:, ["tconst", "titleType"]]

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


In [15]:
df = df1.merge(df3,
              how="left",
              left_on="titleId",
              right_on="tconst")

df = df.loc[df["titleType"] == "movie", ["titleId", "title", "region", "isOriginalTitle"]]

In [16]:
# zamieniamy wszystkie tytuły na lowercase i dekodujemy narodowe znaki
df["title"] = df["title"].astype(str).str.lower().apply(lambda x: codecs.encode(str(x), 'translit/short'))


# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[df["isOriginalTitle"] == 0].merge(
    df.loc[df["isOriginalTitle"] == 1],
    on="titleId",
    how="left",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "region_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

In [17]:
df_think.loc[df_think["titleId"] == 'tt0050634']

,titleId,title,region
273518,tt0050634,letyat zhuravli,EC


In [18]:
# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
disambig = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count()["region"] == 1

df_with_known_countries = df_think.loc[df_think["titleId"].isin(disambig.index[disambig == True])]
df_with_known_countries

,titleId,title,region
4,tt0000147,the corbett-fitzsimmons fight,US
6,tt0000502,bohemios,ES
15,tt0000591,l'enfant prodigue,FR
17,tt0000615,robbery under arms,AU
21,tt0000630,amleto,IT
...,...,...,...
2629074,tt9916620,the copeland case,US
2629075,tt9916622,rodolpho teophilo - o legado de um pioneiro,BR
2629076,tt9916680,de la ilusion al desconcierto: cine colombiano...,CO
2629077,tt9916706,dankyavar danka,IN


In [103]:
path_to_file = "data/title.ratings.tsv.gz"
df7 = load_to_dataframe(path_to_file)

df_analysis = df_with_known_countries.merge(df7,
                                            how="left",
                                            left_on="titleId",
                                            right_on="tconst").drop(columns=["tconst"])
df_analysis = df_analysis.dropna()

In [51]:
# wspieramy rozpoznawanie języka przez bibliotekę 'langdetect'
def mydetect(x):
    try:
        return detect(x).upper()
    except:
        return pd.NA

In [104]:
# usuwamy dziwne filmy z Ekwadoru
df_EC = df_analysis.loc[df_analysis["region"] == 'EC'].copy()
df_EC.loc[:, "detected"] = df_EC["title"].apply(mydetect)

def myfunc_EC(x):
    if x[5] != 'ES':
        x[2] = x[5]
    return x

false_EC = df_EC.loc[df_EC["detected"] != 'ES']["titleId"]
df_EC = df_EC.apply(myfunc_EC, axis=1).drop(columns=["detected"])

df_analysis.loc[df_EC.index] = df_EC

,titleId,title,region,averageRating,numVotes,detected
5775,tt0015039,kinoglaz,EC,6.9,1205.0,HR
5989,tt0015361,stachka,EC,7.6,8675.0,PL
6869,tt0016847,faust: eine deutsche volkssage,EC,8.1,16712.0,DE
6983,tt0017048,kurutta ippeji,EC,7.4,4616.0,FI
8152,tt0019286,potomok chingis-khana,EC,7.0,2322.0,TL
...,...,...,...,...,...,...
447419,tt8777758,montevideo,EC,6.6,52.0,ES
448084,tt8825182,sacachun,EC,8.5,15.0,ES
450352,tt8999974,madre luna,EC,8.5,6.0,RO
452730,tt9182458,torero,EC,7.2,5.0,NO


In [131]:
# usuwamy dziwne filmy z Turcji
df_TR = df_analysis.loc[df_analysis["region"] == 'TR'].copy()
df_TR.loc[:, "detected"] = df_TR["title"].apply(mydetect)
df_TR = df_TR.dropna()

def myfunc_TR(x):
    if x[5] != 'TR':
        x[2] = x[5]
    return x

false_TR = df_TR.loc[df_TR["detected"] != 'TR']["titleId"]
df_TR = df_TR.apply(myfunc_TR, axis=1).drop(columns=["detected"])

df_analysis.loc[df_TR.index] = df_TR

Pytanie: jaką miarę przyłożyć do oceny jakości?
Pomysły:
1. Sortować malejąco po ocenie
2. Sortować malejąco po ocenie, ale odfiltrować filmy z mniej niż $K$ (parametr) ocenami
3. Stworzyć własny indeks, np. $\text{rating} = \text{averageRating}\cdot \text{numVotes}$

Pomysł 1 - nie działa:

In [132]:
df_analysis.sort_values(by="averageRating", ascending=False).head(10)

,titleId,title,region,averageRating,numVotes
214931,tt15108182,ashes to ashes,US,10.0,7.0
453991,tt9303810,"la nueve, heroes en el olvido",ES,10.0,22.0
304879,tt27598401,blackest darkness,US,10.0,11.0
251856,tt19498470,doctor/patient,US,10.0,6.0
440273,tt8215820,are we making a film? behind the scenes: follo...,IE,10.0,10.0
312313,tt28300527,divisible,US,10.0,7.0
313994,tt28488638,debate wars,US,10.0,8.0
364389,tt3823306,the act of writing,US,10.0,10.0
314488,tt28514938,murivu,IN,10.0,350.0
273662,tt22175036,relentless ride,US,10.0,12.0


Pomysł 2 - działa nieźle 

In [135]:
K = 10000
N = 1000
df_analysis.loc[df_analysis["numVotes"] > K].sort_values(
    by="averageRating", ascending=False
).head(N).loc[:, ["titleId", "region"]].groupby(by="region").count()

,titleId
region,
AE,1
AF,1
BR,1
CA,10
CN,2
CY,3
DE,16
DK,11
EC,1


Pomysł 3 -

In [29]:
df_analysis.loc[:, ["rating"]] = df_analysis["averageRating"] * np.log(df_analysis["numVotes"])